## Modelagem exploratória

In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from sistema_antifraude import config
from sistema_antifraude.modelagem.metricas import calcular_metricas
from sistema_antifraude.modelagem.serializacao import salvar_tudo

In [ ]:
df = pd.read_csv(os.path.join(config.DATA_PATH, "processed", "dados_processados.csv"))

In [ ]:
features = [
    'distancia_segundos_entre_acessos',
    'is_dispositivo_anterior_usuario',
    'pais_ip_latlong_concordancia',
    'horario_suspeito',
    'distancia_latlong_anterior'
]
target = 'is_fraude'
X = df[features].copy()
y = df[target].copy()

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size = 0.3, 
    shuffle = True,
    stratify = y,
    random_state = 42
)

In [ ]:
smote = SMOTE(random_state = 42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
y_train_smote.value_counts(normalize = True)

## Árvore de Decisão

In [ ]:
dtc_parametros = {
    'max_depth': 3,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'random_state': 42
}

dtc = DecisionTreeClassifier(**dtc_parametros)
dtc.fit(X_train_smote, y_train_smote)
dtc_y_pred = dtc.predict(X_test)

dtc_metricas = calcular_metricas(y_test, dtc_y_pred)
dtc_metricas

In [ ]:
matriz_dtc = confusion_matrix(y_test, dtc_y_pred, normalize = 'pred')
sns.heatmap(
    matriz_dtc, 
    annot = True, 
    fmt = '.2%', 
    cmap = 'YlOrBr',
    linewidths = 1,
    cbar = False, 
    xticklabels = ['Não Fraude', 'Fraude'], 
    yticklabels = ['Não Fraude', 'Fraude'],
    vmin = 0,
    vmax = 1,
)
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de confusão - Árvore de Decisão')
plt.savefig(
    os.path.join(config.PLOTS_PATH, "matriz_confusao_arvore_decisao.png"), 
    bbox_inches = 'tight',
    dpi = 300
)
plt.show()

In [ ]:
from sklearn import tree
plt.figure(figsize = (20, 10))
tree.plot_tree(dtc, 
    filled = True,
    rounded = True,
    impurity = False,
    proportion = True,
    fontsize = 14, 
    feature_names = [
        'Variação de tempo entre acessos (s)',
        'Dispositivo utilizado anteriormente',
        'País IP e Latitude/Longitude coincidem',
        'Horário suspeito',
        'Distância geográfica entre acessos (km)'
    ], 
    class_names= ['Não Fraude', 'Fraude']
)
plt.savefig(os.path.join(config.PLOTS_PATH, 'arvore_de_decisao.png'), dpi = 300)
plt.show()

## Regressão Logística

In [ ]:
lr = LogisticRegression(max_iter = 1000, random_state = 42)
scaler = StandardScaler()
X_train_smote_scaled = scaler.fit_transform(X_train_smote)
lr.fit(X_train_smote_scaled, y_train_smote)

X_test_scaled = scaler.transform(X_test)
lr_y_pred = lr.predict(X_test_scaled)
lr_metricas = calcular_metricas(y_test, lr_y_pred)
lr_metricas

In [ ]:
matriz_lr = confusion_matrix(y_test, lr_y_pred, normalize = 'pred')
sns.heatmap(
    matriz_lr, 
    annot = True, 
    fmt = '.2%', 
    cmap = 'YlOrBr',
    linewidths = 1, 
    cbar = False, 
    xticklabels = ['Não Fraude', 'Fraude'], 
    yticklabels = ['Não Fraude', 'Fraude']
)
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de confusão - Regressão Logística')
plt.savefig(
    os.path.join(config.PLOTS_PATH, "matriz_confusao_regressao_logistica.png"), 
    bbox_inches = 'tight',
    dpi = 300
)
plt.show()

## Dummy Classifier

In [ ]:
dc = DummyClassifier()
dc.fit(X_train_smote, y_train_smote)

y_predito_dummy = dc.predict(X_test)
dc_metricas = calcular_metricas(y_test, y_predito_dummy)
dc_metricas

In [ ]:
matriz_dummy = confusion_matrix(y_test, y_predito_dummy, normalize = 'pred')
sns.heatmap(
    matriz_dummy, 
    annot = True, 
    fmt = '.2%', 
    cmap = 'YlOrBr',
    linewidths = 1,
    cbar = False, 
    xticklabels = ['Não Fraude', 'Fraude'], 
    yticklabels = ['Não Fraude', 'Fraude']
)
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de confusão')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6), dpi = 300)

TICK_LABELS = ['Não Fraude', 'Fraude']
PADDING_TITLE = 24
PADDING_LABEL = 32
FONTSIZE_TICKS = 14
FONTSIZE_LABEL = 20

sns.heatmap(
    matriz_dummy, 
    annot = True, 
    fmt = '.2%',
    cmap = 'YlOrBr',
    annot_kws = { 'fontsize': 16 },
    linewidth = 1,
    cbar = False,
    ax = axes[0]
)
axes[0].set_title('Dummy', pad = PADDING_TITLE, fontsize = 20, fontweight = 'bold')
axes[0].set_ylabel('Real', labelpad = PADDING_LABEL, fontsize = FONTSIZE_LABEL)
axes[0].set_yticklabels(TICK_LABELS, fontsize = FONTSIZE_TICKS, rotation = 0)
axes[0].set_xticklabels(TICK_LABELS, fontsize = FONTSIZE_TICKS)

sns.heatmap(
    matriz_dtc,
    annot = True,
    fmt = '.2%',
    cmap = 'YlOrBr',
    annot_kws = { 'fontsize': 16 },
    linewidth = 1,
    yticklabels = False,
    cbar = False,
    ax=axes[1]
)
axes[1].set_title('Árvore de Decisão', pad = PADDING_TITLE, fontsize = 20, fontweight = 'bold')
axes[1].set_xlabel('Previsto', labelpad = PADDING_LABEL, fontsize = FONTSIZE_LABEL)
axes[1].set_xticklabels(TICK_LABELS, fontsize = FONTSIZE_TICKS)

sns.heatmap(
    matriz_lr, 
    annot = True,
    fmt = '.2%',
    cmap = 'YlOrBr',
    annot_kws = { 'fontsize': 16 },
    linewidth = 1,
    yticklabels = False,
    cbar = True,
    ax=axes[2]
)
axes[2].set_title('Regressão Logística', pad = PADDING_TITLE, fontsize = 20, fontweight = 'bold')
axes[2].set_xticklabels(TICK_LABELS, fontsize = FONTSIZE_TICKS)

plt.tight_layout()
plt.subplots_adjust(wspace = 0.15)
plt.savefig(os.path.join(config.PLOTS_PATH, 'matrizes.png'), bbox_inches = 'tight', dpi = 300)
plt.show()